In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [4]:
train = pd.read_csv('train.csv',index_col= 'PassengerId')
train['Age'].dtype

dtype('float64')

In [5]:
train.shape
test = pd.read_csv('test.csv',index_col= 'PassengerId')
test.head()


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
train.isnull().sum()
test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

In [7]:
import missingno as mn
mn.matrix(train)

ModuleNotFoundError: No module named 'missingno'

In [8]:
# Data Preprocessing
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan , strategy='median')

Age_2 = train['Age'].values.reshape(1,-1)
Age_2 = Age_2.T
Age_2 = imputer.fit_transform(Age_2)
train['Age_2'] = Age_2
train.head()

Age_test = test['Age'].values.reshape(1,-1)
Age_test = Age_test.T
Age_test = imputer.fit_transform(Age_test)
test['Age_2'] = Age_test
test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_2
PassengerId,,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,34.5
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,47.0
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,62.0
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,27.0
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,22.0


In [9]:
train.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
Age_2         0
dtype: int64

In [10]:
# Taking care of the embarked column
# lets see the values in Embarked column

train['Embarked'].value_counts() # As we see that 'S' value is very frequent, we replace the null with 'S'
train.Embarked.fillna('S',inplace = True)

In [11]:
#Any other missing value in test data set
test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
Age_2         0
dtype: int64

In [12]:
test['Fare'].fillna(test['Fare'].mean(),inplace = True)
#test.isnull().sum()

In [13]:
# We drop cabin column as it has nearly 80% in NA
print(train['Cabin'].isnull().sum()/len(train['Cabin']))
test['Cabin'].isnull().sum()/len(test['Cabin'])

0.7710437710437711


0.7822966507177034

In [14]:
train.drop(['Age','Cabin'], inplace = True, axis = 1)
test.drop(['Age','Cabin'], inplace = True, axis = 1)

In [15]:
train.head()

,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Embarked,Age_2
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,1,0,A/5 21171,7.2500,S,22.0
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,PC 17599,71.2833,C,38.0
3,1,3,"Heikkinen, Miss. Laina",female,0,0,STON/O2. 3101282,7.9250,S,26.0
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,113803,53.1000,S,35.0
5,0,3,"Allen, Mr. William Henry",male,0,0,373450,8.0500,S,35.0


In [16]:
test.head() 

,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Embarked,Age_2
PassengerId,,,,,,,,,
892,3,"Kelly, Mr. James",male,0,0,330911,7.8292,Q,34.5
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,1,0,363272,7.0000,S,47.0
894,2,"Myles, Mr. Thomas Francis",male,0,0,240276,9.6875,Q,62.0
895,3,"Wirz, Mr. Albert",male,0,0,315154,8.6625,S,27.0
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,1,1,3101298,12.2875,S,22.0


In [17]:
# Removing outliers

Q1 = train['Fare'].quantile(0.25)
Q3 = train['Fare'].quantile(0.75)

IQR = Q3-Q1
print(IQR)

train[(train['Fare'] < (Q1 - 1.5 * IQR)) | (train['Fare'] > (Q3 + 1.5 * IQR))]['Fare'].max()
train['Fare'].median

# We are not deleting any outliers as most is dependent on catagorial data and we dont have any catagorical outliers

23.0896


<bound method Series.median of PassengerId
1       7.2500
2      71.2833
3       7.9250
4      53.1000
5       8.0500
        ...   
887    13.0000
888    30.0000
889    23.4500
890    30.0000
891     7.7500
Name: Fare, Length: 891, dtype: float64>

In [18]:
# we see that there is an outlier in fare that has to be removed
# so we dont consider any fare abover 400
train['Fare'].isnull().sum()
test.isnull().sum()

Pclass      0
Name        0
Sex         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    0
Age_2       0
dtype: int64

In [19]:
train = pd.get_dummies(train,columns  =['Sex','Embarked','Pclass'], drop_first = True)
test = pd.get_dummies(test,columns  =['Sex','Embarked','Pclass'], drop_first = True)

In [20]:
train.head()

,Survived,Name,SibSp,Parch,Ticket,Fare,Age_2,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
PassengerId,,,,,,,,,,,,
1,0,"Braund, Mr. Owen Harris",1,0,A/5 21171,7.2500,22.0,1,0,1,0,1
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,0,PC 17599,71.2833,38.0,0,0,0,0,0
3,1,"Heikkinen, Miss. Laina",0,0,STON/O2. 3101282,7.9250,26.0,0,0,1,0,1
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,0,113803,53.1000,35.0,0,0,1,0,0
5,0,"Allen, Mr. William Henry",0,0,373450,8.0500,35.0,1,0,1,0,1


In [21]:
# Converting DataFrames to arrays for ML Algorithms
X_train = train.iloc[:,5:].values
y_train = train.iloc[:,0].values
x_test = test.iloc[:,4:].values

#Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
x_test = sc.transform(x_test)

In [44]:
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

classifierKNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifierSVML = SVC(kernel = 'linear')
classifierSVMR = SVC(kernel = 'rbf')
classifierRF = RandomForestClassifier(n_estimators = 50, criterion = 'entropy')
classifierNB = GaussianNB()

score_NB = []
score_rf =[]
score_svml = []
score_svmr = []
score_KNN = []

def get_score(model,X_train,X_test,y_train,y_test):
    model.fit(X_train,y_train)
    return model.score(X_test,y_test) 

folds = StratifiedKFold(n_splits = 10)
for train_index,test_index in folds.split(X_train,y_train):
    X_train_1,y_train_1,X_test_1,y_test_1 = X_train[train_index],y_train[train_index],X_train[test_index],y_train[test_index]
    score_NB.append(get_score(classifierNB,X_train_1,X_test_1,y_train_1,y_test_1))
    score_KNN.append(get_score(classifierKNN,X_train_1,X_test_1,y_train_1,y_test_1))
    score_rf.append(get_score(classifierRF,X_train_1,X_test_1,y_train_1,y_test_1))
    score_svml.append(get_score(classifierSVML,X_train_1,X_test_1,y_train_1,y_test_1))
    score_svmr.append(get_score(classifierSVMR,X_train_1,X_test_1,y_train_1,y_test_1))
    
from statistics import mean
print("Score Naive Bayes   : {}".format(mean(score_NB)))
print("Score K Nearest N   : {}".format(mean(score_KNN)))
print("Score SVM Linear    : {}".format(mean(score_svml)))
print("Score SVM Radial    : {}".format(mean(score_svmr)))
print("Score Random Forest : {}".format(mean(score_rf)))

Score Naive Bayes   : 0.7867915106117354
Score K Nearest N   : 0.8148689138576779
Score SVM Linear    : 0.786729088639201
Score SVM Radial    : 0.8260299625468165
Score Random Forest : 0.8204993757802747


In [35]:
classifierSVMR.fit(X_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [37]:
y_test = classifierSVMR.predict(x_test)

In [40]:
test.reset_index(level=0, inplace=True)

In [41]:
y_test_2 = pd.Series(y_test)
Final = pd.DataFrame(test['PassengerId'])
Final['Survived'] = y_test

In [42]:
Final.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [43]:
Final.to_csv('Titanic_with SVMR_2.csv', index=False)

In [200]:
test

,PassengerId,Name,SibSp,Parch,Ticket,Fare,Age_2,Sex_male,Embarked_Q,Embarked_S,Pclass_2,Pclass_3
0,892,"Kelly, Mr. James",0,0,330911,7.8292,34.5,1,1,0,0,1
1,893,"Wilkes, Mrs. James (Ellen Needs)",1,0,363272,7.0000,47.0,0,0,1,0,1
2,894,"Myles, Mr. Thomas Francis",0,0,240276,9.6875,62.0,1,1,0,1,0
3,895,"Wirz, Mr. Albert",0,0,315154,8.6625,27.0,1,0,1,0,1
4,896,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,1,3101298,12.2875,22.0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,"Spector, Mr. Woolf",0,0,A.5. 3236,8.0500,27.0,1,0,1,0,1
414,1306,"Oliva y Ocana, Dona. Fermina",0,0,PC 17758,108.9000,39.0,0,0,0,0,0
415,1307,"Saether, Mr. Simon Sivertsen",0,0,SOTON/O.Q. 3101262,7.2500,38.5,1,0,1,0,1
416,1308,"Ware, Mr. Frederick",0,0,359309,8.0500,27.0,1,0,1,0,1
